In [6]:
# Importing Required Packages
import tensorflow as tf
import pandas
import numpy as np
import cv2
import os.path
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image as im
from skimage.color import rgb2lab, lab2rgb
from skimage.io import imsave
import io

In [4]:
# Prototext, models, points provided by Caffe and https://github.com/PySimpleGUI/PySimpleGUI-Photo-Colorizer
points = r'Caffe Model/pts_in_hull.npy'
prototxt = r'Caffe Model/colorization_deploy_v2.prototxt'
model = r'Caffe Model/colorization_release_v2.caffemodel'

# Loading the model, please ensure to download the model from https://www.dropbox.com/s/dx0qvhhp5hbcx7z/colorization_release_v2.caffemodel?dl=1 and copy it in the Caffe Model folder
# Model has to be downloaded due to its size being too large for github
net = cv2.dnn.readNetFromCaffe(prototxt, model)     
pts = np.load(points)

# Code from PySimple GUI that adds the cluster centers as 1x1 convolutions to the model
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

# Defining function for image colorization
def colorize_image(image_filename=None):
    # Loading input image, scaling and coverting to labspace
    image = cv2.imread(image_filename) 
    scaled_image = image.astype("float32") / 255.0
    lab = rgb2lab(scaled_image)

    # Resizing the Lab image to 224x224 (the dimensions the colorization network accepts), split channels, extract the 'L' channel, and then perform mean centering
    resized = cv2.resize(lab, (224, 224))
    L = cv2.split(resized)[0]
    L -= 50

    # Passing the L channel through the network which will *predict* the 'a' and 'b' channel values
    net.setInput(cv2.dnn.blobFromImage(L))
    ab = net.forward()[0, :, :, :].transpose((1, 2, 0))

    # Resizing the predicted 'ab' volume to the same dimensions as our input image
    ab = cv2.resize(ab, (image.shape[1], image.shape[0]))

    # Grabbing the 'L' channel from the *original* input image (not the resized one) and concatenate the original 'L' channel with the predicted 'ab' channels
    L = cv2.split(lab)[0]
    colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)

    # Converting the output image from the Lab color space to RGB, then clip any values that fall outside the range [0, 1]
    colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
    colorized = np.clip(colorized, 0, 1)

    # The colorized image is represented as a floating point data type in the range [0, 1] -- let's convert to an unsigned 8-bit integer representation in the range [0, 255]
    colorized = (255 * colorized).astype("uint8")
    return colorized


In [20]:
# Colorizing the image using the function above and saving them in the folder

for filename in os.listdir("Caffe Model Input Images/"):
    colorized = colorize_image("Caffe Model Input Images/"+filename)
    bytedata=cv2.imencode('.png', colorized)[1].tobytes()
    imagearray = np.array(im.open(io.BytesIO(bytedata))) 
    imsave("Caffe Model Result Images/"+filename+".png", imagearray)